In [ ]:
#-- Import libraries ------------------------------------------------------------------------------------------------------
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize -------------------------------------------------------------------------------------------------------------
batch_size = 2000
batch_number = 0
N_BACKS = 5
darkening_factor = 0.2

backgrounds_path = "/kaggle/input/n-drone-clear-backs/backgrounds/"  
drones_path = "/kaggle/input/n-drone-clear-drones/drones/"  
output_path = f"dataset_no_scale_part_{batch_number+1}"  

os.makedirs(output_path, exist_ok=True)
os.makedirs(os.path.join(output_path, "images"), exist_ok=True)
os.makedirs(os.path.join(output_path, "labels"), exist_ok=True)
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- get all backgrounds and drones as 2 lists ------------------------------------------------------------------------------
backgrounds = [os.path.join(backgrounds_path, f) for f in os.listdir(backgrounds_path)]            
drones = [os.path.join(drones_path, f) for f in os.listdir(drones_path)]

backgrounds = sorted(backgrounds)
drones = sorted(drones)

print(f'number of background images: {len(backgrounds)}')
print(f'number of drone images: {len(drones)}')
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- set batch ---------------------------------------------------------------------------------------------------------------
total_drones = len(drones)  
if total_drones == 0:
    raise ValueError("No drones found in the dataset!")

drones_per_batch = max(1, batch_size // N_BACKS)  
num_batches = (total_drones + drones_per_batch - 1) // drones_per_batch  

start_idx = batch_number * drones_per_batch
end_idx = min(start_idx + drones_per_batch, total_drones)  

print(f"Processing batch {batch_number + 1}/{num_batches}, drones {start_idx} to {end_idx - 1}")
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to add drone object to background image -----------------------------------------------------------------------
def add_object_to_background(background_img, object_img, x, y):
    
    object_h, object_w = object_img.shape[:2]
    
    #-- Crop the region of interest (ROI) from the background with the same size as the object --
    roi = background_img[y:y+object_h, x:x+object_w]

    #-- Extract the alpha channel as a mask --
    object_rgb = object_img[:, :, :3]  #-- RGB channels (BGR)
    mask = object_img[:, :, 3] / 255.0  #-- Normalize alpha channel to the range [0,1]

    #-- Blend the object with the background based on the alpha mask --
    for c in range(3):  # For each color channel (BGR)
        roi[:, :, c] = (1 - mask) * roi[:, :, c] + mask * object_rgb[:, :, c]

    # --Place the modified region back into the original background--
    background_img[y:y+object_h, x:x+object_w] = roi
    return background_img
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Function to Displaye Result of Blending Drone and background -----------------------------------------------------------
def show_image_with_bbox(image, x_center, y_center, w_norm, h_norm, title):
    img_with_bbox = image.copy()
    h, w, _ = img_with_bbox.shape      
    
    bbox_width = int(w_norm * w)
    bbox_height = int(h_norm * h)
    x_min = int((x_center * w) - (bbox_width / 2))
    y_min = int((y_center * h) - (bbox_height / 2))
    x_max = x_min + bbox_width
    y_max = y_min + bbox_height
    
    color = (0, 0, 255)  
    thickness = 2 
    cv2.rectangle(img_with_bbox, (x_min, y_min), (x_max, y_max), color, thickness)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(cv2.cvtColor(img_with_bbox, cv2.COLOR_BGR2RGB)) 
    plt.axis("off")
    plt.title(title)
    plt.show()
#---------------------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Run --------------------------------------------------------------------------------------------------------------------
count = 0  

for i in range(start_idx+50, end_idx):
    drone = drones[i]
    selected_backgrounds = random.sample(backgrounds, min(N_BACKS, len(backgrounds)))  
    
    for j, background in enumerate(selected_backgrounds):
        if count >= batch_size:
            break  

        if count % 100 == 0:
            print(f'Processing {count + 1}th item: {drone} with {background}')

        background_path = selected_backgrounds[j]
        drone_path = drone        

        background_img = cv2.imread(background_path)        
        drone_img = cv2.imread(drone_path, cv2.IMREAD_UNCHANGED)

        plt.figure(figsize=(3, 3))
        plt.imshow(cv2.cvtColor(drone_img, cv2.COLOR_BGR2RGB)) 
        plt.axis("off")
        plt.show()

        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(background_img, cv2.COLOR_BGR2RGB)) 
        plt.axis("off")
        plt.show()
            
        if background_img is None or drone_img is None:
            print(f'NONE: {background_path}\n{drone_path}')
            continue   
        
            
        #-- Darken drone_img --        
        if drone_img.shape[-1] == 4:            
            #-- Split channels --
            bgr_channels = drone_img[:, :, :3]  #-- Extract RGB/BGR channels
            alpha_channel = drone_img[:, :, 3]  #-- Extract the alpha channel
        
            #-- Create a mask where alpha > 0 (non-transparent areas) --
            mask = alpha_channel > 0  
        
            #-- Darken only the non-transparent areas (scaling towards black) --             
            bgr_channels[mask] = (bgr_channels[mask].astype(np.float32) * darkening_factor).astype(np.uint8)
        
            #-- Merge back with the unchanged alpha channel --
            drone_img = cv2.merge((bgr_channels, alpha_channel))
        
        else: #-- If there's no alpha channel, just darken the whole image            
            drone_img = (drone_img.astype(np.float32) * darkening_factor).astype(np.uint8)        

        background_h, background_w, _ = background_img.shape
        drone_h, drone_w, _ = drone_img.shape

        #-- set scale factor based on ratio --
        ratio_w = drone_w / background_w  
        ratio_h = drone_h / background_h  
        ratio = min(ratio_w, ratio_h)   
        ratio = round(ratio,2)  

        if ratio <0.04:
            continue
        elif ratio>=0.04 and ratio<=0.05:
            scale_factor = round(random.uniform(0.8,0.9), 2)  
        elif ratio>0.05 and ratio<=0.1:
            scale_factor = round(random.uniform(0.5,0.7), 2)  
        elif ratio>0.1 and ratio<=0.2:
            scale_factor = round(random.uniform(0.4,0.6), 2)  
        elif ratio>0.2 and ratio<=0.3:
            scale_factor = round(random.uniform(0.3,0.5), 2)  
        elif ratio>0.3 and ratio<=0.4:
            scale_factor = round(random.uniform(0.2,0.4), 2)  
        else: # >0.4
            scale_factor = round(random.uniform(0.1,0.2), 2)  

        new_w = int(drone_w * scale_factor)
        new_h = int(drone_h * scale_factor)
        drone_img = cv2.resize(drone_img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        drone_h, drone_w, _ = drone_img.shape            

        #-- Select a random location to place the object --
        x_min = random.randint(0, background_w - drone_w)
        y_min = random.randint(0, background_h - drone_h)
        x_max = x_min + drone_w
        y_max = y_min + drone_h        
            
        #-- Insert the object image onto the background with transparency --
        background_with_obj = background_img.copy()
        background_with_obj = add_object_to_background(background_with_obj,
                                                           drone_img,
                                                           x_min, y_min)
    
        #-- Calculate the Bounding Box in YOLO format --
        x_center = (x_min + x_max) / 2 / background_w
        y_center = (y_min + y_max) / 2 / background_h
        w_norm = drone_w / background_w
        h_norm = drone_h / background_h
    
        #-- Save the result image --
        output_img_name = f"{os.path.basename(drone_path).split('.')[0]}_{scale_factor}_{os.path.basename(background_path).split('.')[0]}.jpg"
        output_img_path = os.path.join(output_path, "images", output_img_name)
        os.makedirs(os.path.dirname(output_img_path), exist_ok=True)  
        cv2.imwrite(output_img_path, background_with_obj)
    
        #-- Save the YOLO file --
        output_txt_name = output_img_name.replace(".jpg", ".txt")
        output_txt_path = os.path.join(output_path, "labels", output_txt_name)
        os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
        with open(output_txt_path, "w") as f:
            f.write(f"0 {x_center} {y_center} {w_norm} {h_norm}\n")           
    
        #-- Display the final image with the bounding box --
        title = output_img_name
        show_image_with_bbox(background_with_obj, x_center, y_center, w_norm, h_norm, title)
            
        count += 1
        
        if count >= batch_size:
            break  
    if count >= batch_size:
        break
#---------------------------------------------------------------------------------------------------------------------------